<span style="font-size: 100%; ">論文“井元佑介, 超双対数に基づく高階微分計算手法の開発と応用, 応用数理, 2024.”における超双対数の行列表示を用いた微分計算の計算例</span>

# 超双対数単位と超双対数空間上の関数の定義 

In [1]:
# coding: utf-8
import numpy as np
import math
from functools import reduce

def Setting(n):
    global nHDN
    nHDN = n
    global HDN
    E_11 = np.array([[0, 1], [0, 0]], dtype=float)
    I_2 = np.eye(2)
    HDN = np.empty(nHDN, dtype=object)
    for i in range(nHDN):
        kron_list = [E_11 if j == nHDN - i - 1 else I_2 for j in range(nHDN)]
        HDN[i] = reduce(np.kron, kron_list)
    global ID
    ID = np.array(np.eye(2**nHDN))

def Deriv(Mat, k):
    if 0 <= k <= nHDN:
        return Mat[0,2**k-1]
    else:
        return "error"

def prod(Mat1,Mat2):
    return np.dot(Mat1,Mat2)

def const():
    return ID


def Hini(x):
    return  x * ID + np.sum(HDN,axis=0)

def Hsin(Mat):
    a = Mat[0,0]
    Q = Mat-a*ID
    val = np.sin(a)*ID
    for i in range(nHDN):
        val += (Q**(i+1))/math.factorial(i+1)*(
            (-1)**(((i+1)//2)%4)*((i+2)%2)*math.sin(a)
            +(-1)**((i//2)%4)*(i+1)%2)*math.cos(a)
    return  val

def Hcos(Mat):
    a = Mat[0,0]
    Q = Mat-a*ID
    val = math.cos(a)*ID
    for i in range(nHDN):
        val += (Q**(i + 1)) * (
                    pow(-1, ((i + 1) // 2) % 4) * ((i + 2) % 2) * math.cos(Mat[0, 0]) + pow(-1, ((i + 2) // 2) % 4) * (
                        (i + 1) % 2) * math.sin(Mat[0, 0])) / math.factorial(i + 1)
    return  val


def Hexp(Mat):
    val = math.exp(Mat[0, 0]) * ID
    for i in range(nHDN):
        val += ((Mat - Mat[0, 0] * ID) ** (i + 1)) * math.exp(Mat[0, 0]) / math.factorial(i + 1)
    return  val


def Hlog(Mat):
    val = math.log(Mat[0, 0]) * ID
    for i in range(nHDN):
        val += ((Mat - Mat[0, 0] * ID) ** (i + 1)) * (pow(-1, i)) * (Mat[0, 0] ** (-i - 1)) / (i + 1)
    return  val


def Hpow(Mat, m):
    coef = 1
    val = math.pow(Mat[0, 0], m) * ID
    for i in range(nHDN):
        coef *= m-i
        val += coef*((Mat - Mat[0, 0] * ID) ** (i + 1)) * math.pow(Mat[0, 0], m - i - 1) / math.factorial(i + 1)
    return  val


def HiniMult(x,k,Jn):
    val = x * ID
    for i in range(nHDN):
        if Jn[i] == k:
            val += HDN[i]
    return  val

# 常備分
$f(x)=\sin(x) \cos(x)$ の $x=0$ での高階微分値の計算

In [2]:
# 微分階数 (超双対数の次数)
n = 10

#　初期設定 (超双対数単位の構成)
Setting(n)

# 関数の定義
def Func(x):
    return Hsin(Hini(x))*Hcos(Hini(x))

# 変数の値
x = 0


# 計算結果
for i in range(n+1):
    print(i,"-derivative(s)=",Deriv(Func(x),i))

0 -derivative(s)= 0.0
1 -derivative(s)= -0.5402372697085075
2 -derivative(s)= 0.0
3 -derivative(s)= 0.0
4 -derivative(s)= 0.0
5 -derivative(s)= 0.0
6 -derivative(s)= 0.0
7 -derivative(s)= 0.0
8 -derivative(s)= 0.0
9 -derivative(s)= 0.0
10 -derivative(s)= 0.0


# 偏微分
## 付録Bの例

$f(x)=e^{\cos(x_1)+x_2^2}$ の $(x_1,x_2)=(1,1)$ 二階偏微分 $\partial_1\partial_2 f(x)$ の計算

In [3]:
# 微分する変数のインデックス
Jn = [1,2] # ∂x1∂x2
n = len(Jn)

#　初期設定 (超双対数単位の構成)
Setting(n)

# 関数の定義
def Func(x1,x2):
    return Hexp(Hcos(HiniMult(x1,1,Jn))+Hpow(HiniMult(x1,2,Jn),2))

# 変数の値
x1 = 1
x2 = 1

行列$\mathcal{T}_2 f(x+e_1\otimes\mathcal{E}_{2,1}+e_2\otimes\mathcal{E}_{2,2})$

In [4]:
Func(x1,x2)

array([[ 4.66600062, -2.30393229, 34.99500463,  0.        ],
       [ 0.        ,  4.66600062,  0.        , 34.99500463],
       [ 0.        ,  0.        ,  4.66600062, -2.30393229],
       [ 0.        ,  0.        ,  0.        ,  4.66600062]])

二階偏微分 $\partial_1\partial_2 f(x) = \mathfrak{M}_{1,4}[\mathcal{T}_2 f(x+e_1\otimes\mathcal{E}_{2,1}+e_2\otimes\mathcal{E}_{2,2})]$ 

In [5]:
Func(x1,x2)[0,3]

0.0

## 10階偏微分の例

$f(x_1,x_2,x_3) = \exp(\sin(\sqrt{x_1}+3x_2)) \log(\cos^2(x_3+0.5))$ の偏微分 $\partial_1^3 \partial_2^5 \partial_3^2 f(x_1,x_2,x_3)$

In [6]:
# 微分する変数のインデックス
Jn = [1,1,1,2,2,2,2,2,3,3] 
n = len(Jn)

#　初期設定 (超双対数単位の構成)
Setting(n)

# 関数の定義
def MultiFunc(x1,x2,x3):
    return prod(Hexp(Hsin(Hpow(HiniMult(x1,1,Jn),1/2)+3.*HiniMult(x2,2,Jn))),Hlog(Hpow(Hcos(HiniMult(x3,3,Jn)),2)+0.5*const()))

# values
x1 = 0.5*math.pi
x2 = 0.25*math.pi
x3 = 2.

# 計算結果
for i in range(n+1):
    print(i,"-partial derivative(s)=",Deriv(MultiFunc(x1,x2,x3),i))


0 -partial derivative(s)= -0.2520792083298674
1 -partial derivative(s)= 0.07323225947950249
2 -partial derivative(s)= 0.0
3 -partial derivative(s)= 0.0
4 -partial derivative(s)= 0.0
5 -partial derivative(s)= 0.0
6 -partial derivative(s)= 0.0
7 -partial derivative(s)= 0.0
8 -partial derivative(s)= 0.0
9 -partial derivative(s)= 0.0
10 -partial derivative(s)= 0.0
